### <span style="color: orange; font-family: 'Press Start 2P',system-ui"> Code : 開啟主檔 </span>

In [20]:
import pandas as pd
import numpy as np
def file(file_name, sheet_name):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    return df

df_fs = file("上市櫃公司.xlsx",sheet_name="Sheet1")

### 用途 : 對比TEJ上市櫃基本資料 2024年報 2025第一季資料 ，找出未重疊的證券代碼

In [1]:
import pandas as pd
df=pd.read_excel('FirmCode_years.xlsx')
df["2024年年報"]=df["2024年年報"].fillna(0).astype(int)
df["2025年第一季季報"]=df["2025年第一季季報"].fillna(0).astype(int)
df.to_excel('FirmCode_years.xlsx', index=False)

In [25]:
try:
    df = pd.read_excel('FirmCode_years.xlsx')
    df["2024年年報"] = df["2024年年報"].astype(int)
    print("File loaded successfully")
except PermissionError as e:
    print(f"Permission error: {e}. Ensure the file is not open elsewhere.")
except Exception as e:
    print(f"Other error: {e}")


File loaded successfully


### 優先使用broadcasting

#### 1.numpy setxor1d = 自動展平陣列並找出不重複的元素，且回傳結果會自動排序

#### 2. numpy setdiff =  找出跟array 1相比，array2 沒有的缺失值

#### 3.結果 --> 3717 聯家投控

In [3]:
import numpy as np
stock_basic=df["基本資料"].values.flatten()
stock_2024=df["2024年年報"].values.flatten()
stock_2025=df["2025年第一季季報"].values.flatten()

print(type(stock_basic))
non_common=np.setdiff1d(stock_basic, stock_2024)


<class 'numpy.ndarray'>


In [4]:
non_common_2025=np.setdiff1d(stock_basic, stock_2025)
non_common_24_25=np.setdiff1d(stock_2024, stock_2025)
print(non_common_2025)
print(len(non_common_2025))
print(non_common_24_25)

[  3717   6620   7777   9103   9136 910322 910861 911608 911868 912000]
10
[  6620   7777   9103   9136 910322 910861 911608 911868 912000]


In [5]:
import pandas as pd
df_march=pd.read_excel('2025年第一季季報資本總額.xlsx',sheet_name="march")
df_march_codes=df_march["證期會代碼"].values.flatten()
diff_march=np.setdiff1d(non_common_2025, df_march_codes)
print(diff_march)

[  3717   6620   7777   9103   9136 910322 910861 911608 911868 912000]


In [6]:
import pandas as pd
df_2025fs=pd.read_excel('2025_第一季財務狀況.xlsx')
df_2025fs_codes=df_2025fs["避險之金融資產－流動"].fillna(0).astype(int)
print(df_2025fs_codes)
df_2025fs.to_excel('2025_第一季財務狀況.xlsx', index=False)

0       0
1       0
2       0
3       0
4       0
       ..
1911    0
1912    0
1913    0
1914    0
1915    0
Name: 避險之金融資產－流動, Length: 1916, dtype: int64


### 槓桿比 =  總負債 / 總權益

In [8]:
import pandas as pd
def file(file_name, sheet_name):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    return df

df_fs = file("上市櫃公司.xlsx",sheet_name="Sheet1")
df_25s1=file("2025_第一季財務狀況.xlsx",sheet_name="Sheet1")
df_fs["槓桿比"]=df_25s1["負債總額"]/(df_25s1["資產總額"]-df_25s1["負債總額"])
df_fs.to_excel('上市櫃公司.xlsx', index=False)


In [ ]:
df_minus1=df_fs[df_fs["槓桿比"]>6]
print(df_fs["槓桿比"].describe())

count    1916.000000
mean        1.172158
std         2.179953
min         0.008896
25%         0.380056
50%         0.697676
75%         1.205298
max        43.469290
Name: 槓桿比, dtype: float64


### 現金比率 = (現金及約當現金+FVPL金融資產-流動+FV0CI金融資產-流動+AC金融資產-流動+避險金融資產-流動) / 資產總額

In [22]:
df_fs["現金比率"]=(df_25s1["現金及約當現金"]+df_25s1["透過損益按公允價值衡量之金融資產－流動"]+df_25s1["透過其他綜合損益按公允價值衡量之金融資產－流動"]+df_25s1["按攤銷後成本衡量之金融資產－流動"]+df_25s1["避險之金融資產－流動"])/df_25s1["資產總額"]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

### 資產報酬率 (先使用2024全年)

### 程式暫時訂為2025s1淨利/2025s1資產總額



In [23]:
df_pl2025=file("2025上市櫃損益表_S1.xlsx",sheet_name="Sheet1")
df_fs["資產報酬率"]=df_pl2025["常續性稅後淨利"]/df_25s1["資產總額"]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

### OI = 營業淨利/總資產 (先使用2024全年)
### 程式暫時訂為2025s1營業利益/2025s1資產總額

In [24]:
df_fs["OI"]=df_pl2025["營業利益"]/df_25s1["資產總額"]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

### EBITDA = 稅前息前折舊攤銷前盈餘/總資產 
### EBIT = 稅前息前盈餘/總資產
1. 盈餘 = 2024年全年度  
2. 總資產 = 2025 S1 之累計數

In [12]:
df_EBIT_2024=file("EBIT_EBITDA.xlsx",sheet_name="Sheet1")
df_fs["EBIT"]=df_EBIT_2024["稅前息前淨利"]/df_25s1["資產總額"]
df_fs["EBITDA"]=df_EBIT_2024["稅前息前折舊前淨利"]/df_25s1["資產總額"]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

### 2. 國際供應鏈與客戶曝險程度(Supply Chain，SC)

### <font color="red"> 之後再做 : 目前尚無2025年度銷貨毛利數據</font>

### 3.ESG --> 可能需使用TESG模組

#### <font color="green"> 中興沒買，借政大or北大 </font>

### 4.公司治理 
#### 董事會規模（Board Size）= 董事會成員總數

In [16]:
df_CG=file("2025年公司治理.xlsx",sheet_name="Sheet1")
df_CG["代號"]=df_CG["代號"].astype(int)
df_CG.to_excel('2025年公司治理.xlsx', index=False)

In [27]:
import numpy as np
non_common_CG_2025=np.setdiff1d(df_fs["代號"].values, df_CG["代號"].values)
non_common_CG_2024=np.setdiff1d(df_CG["代號"].values, df_fs["代號"].values)
print("公司治理模組相較上市櫃資料缺失值:",non_common_CG_2025)
print("上市櫃資料相較公司治理模組缺失值:",non_common_CG_2024)

公司治理模組相較上市櫃資料缺失值: [  3321   9105   9110 911622]
上市櫃資料相較公司治理模組缺失值: [3717 6620 6722 7777 7795]


In [29]:
df_25CG_independence=file("2025年公司獨董人數.xlsx",sheet_name="Sheet1")
df_25CG_independence["代號"]=df_25CG_independence["代號"].astype(int)
df_25CG_independence.to_excel('2025年公司獨董人數.xlsx', index=False)

### <span style="color: white; font-family: 'cwTeXYen '"> 董事長兼總經理 (Dual) = 當公司董事長兼任總經理時，則變數設為1，否則為0 </span>


### <span style="color : pink" >First prompt : 證期會代碼由文字轉數字後，用sort_values由小到大排序 (ascending = True) </span>

In [25]:
import pandas as pd
df_dual_2025=file("2025年董事長兼總經理.xlsx",sheet_name="Sheet1")
df_dual_2025["代碼"]=df_dual_2025["代碼"].astype(int)
df_dual_2025=df_dual_2025.sort_values(by="代碼", ascending=True)
dual_code=pd.unique(df_dual_2025["代碼"].values)
df_dual_2025.to_excel('2025年董事長兼總經理.xlsx', index=False)
print(dual_code)

[1101 1102 1103 ... 9958 9960 9962]


### <span style="color : pink" >Second prompt : 用np.setdiffid分別找出2者資料的相對缺失值 </span>

In [26]:
import numpy as np
df_fs_code=df_fs["代號"].values.flatten()
lost_code_dual=np.setdiff1d(df_fs_code, dual_code)
lost_code_fs=np.setdiff1d(dual_code, df_fs_code)
print(len(dual_code))
print("董事長兼總經理模組相較上市櫃資料缺失值:",lost_code_dual)
print("上市櫃資料相較董事長兼總經理模組缺失值:",lost_code_fs)

1914
董事長兼總經理模組相較上市櫃資料缺失值: [  6907   7717   7749   9105   9110 911622]
上市櫃資料相較董事長兼總經理模組缺失值: [6620 6722 7777 7795]


### <span style="color : pink" >3rd prompt : 鎖定董事長兼總經理資料並匯出  </span>

#### <span style="color : red">Notice : 複數工作表需逐項重新匯入，否則會被洗掉 </spam>

In [27]:
df_dual_2025_CEOplusGM=df_dual_2025[df_dual_2025["身份別"]=="董事長兼總經理"]["代碼"].unique()
print(df_dual_2025_CEOplusGM)
print(type(df_dual_2025_CEOplusGM))
print(len(df_dual_2025_CEOplusGM))
print("比例:",len(df_dual_2025_CEOplusGM)/len(dual_code))

[1240 1294 1301 1321 1323 1341 1342 1410 1413 1437 1454 1465 1468 1524
 1526 1533 1536 1541 1563 1570 1580 1582 1586 1590 1593 1599 1626 1707
 1720 1733 1781 1785 2030 2069 2073 2206 2235 2248 2313 2317 2330 2344
 2353 2359 2360 2368 2377 2380 2383 2388 2393 2401 2458 2459 2461 2462
 2472 2478 2480 2481 2483 2486 2488 2493 2495 2496 2497 2498 2605 2637
 2723 2724 2727 2729 2732 2734 2743 2753 2762 2905 2908 3002 3005 3006
 3014 3015 3025 3026 3030 3031 3036 3047 3049 3052 3060 3062 3141 3149
 3176 3206 3213 3226 3227 3228 3232 3285 3288 3296 3317 3323 3346 3349
 3354 3360 3362 3434 3437 3508 3516 3528 3535 3540 3545 3546 3552 3576
 3581 3591 3597 3631 3675 3687 3693 3714 4111 4114 4121 4148 4161 4188
 4417 4426 4527 4555 4564 4588 4720 4737 4741 4746 4747 4760 4768 4770
 4909 4911 4915 4923 4931 4939 4949 4953 4956 4994 4999 5014 5212 5215
 5222 5228 5230 5251 5263 5274 5289 5353 5425 5426 5432 5460 5464 5478
 5520 5529 5543 5603 6101 6104 6113 6116 6121 6130 6134 6138 6158 6163
 6164 

In [28]:
df_dual_2025_CEOplusGM_sheet=df_dual_2025[df_dual_2025["身份別"]=="董事長兼總經理"]
with pd.ExcelWriter('2025年董事長兼總經理.xlsx', engine='openpyxl') as writer:
    df_dual_2025.to_excel(writer, index=False,sheet_name='Sheet1')
    df_dual_2025_CEOplusGM_sheet.to_excel(writer, index=False,sheet_name='CEOplusGM')

In [29]:
df_dual_2025_NonCEOplusGM=np.setdiff1d(dual_code, df_dual_2025_CEOplusGM)
print(df_dual_2025_NonCEOplusGM)
print(len(df_dual_2025_NonCEOplusGM)+len(df_dual_2025_CEOplusGM))
print(len(df_dual_2025["代碼"].unique()))

[1101 1102 1103 ... 9958 9960 9962]
1914
1914


### 4th prompt : 找出上市櫃資料 v.s 董事長兼總經理的公司之間的共同公司

#### 使用np.intersect1d找出共同的公司代碼 --> 填入1

#### np.intersect1d 的使用 --> return_indices=True=顯示共同數的index
#### <a href="https://ppt.cc/fmgMGx"><img src="https://ppt.cc/fmgMGx@.png" border="0"></a>

#### 從len(df_fs_code_CEOplusGM) = len(arr1) 可知上市櫃資料中公司完全涵蓋了CEOplusGM的公司數
###  Outcome : 無缺失值

In [ ]:
df_fs_code=df_fs["代號"].values.flatten()
df_fs_code_CEOplusGM=df_dual_2025_CEOplusGM_sheet["代碼"].values.flatten()
print(len(df_fs_code_CEOplusGM))
df_fs_dual_1,arr1,arr2=np.intersect1d(df_fs_code,df_fs_code_CEOplusGM,return_indices=True)
print(len(arr1))
if len(arr1)==len(df_fs_code_CEOplusGM):
    print("全部有對應上")
lost_CEOplusGM=np.setdiff1d(df_fs_code_CEOplusGM, df_fs_code)
print("上市櫃公司缺失董事長兼總經理資料代號:",lost_CEOplusGM)

308
308
全部有對應上
上市櫃公司缺失董事長兼總經理資料代號: []


### 再拿非董事長兼總經理的公司資料，與上市櫃公司資料比較，找出缺失值

In [ ]:
df_dual_2025_NonCEOplusGM=np.setdiff1d(dual_code, df_dual_2025_CEOplusGM)
lost_NonCEOplusGM=np.setdiff1d(df_dual_2025_CEOplusGM, df_fs["代號"].values.flatten())
print(len(lost_NonCEOplusGM))

### <span style="color : #59912C"> .loc 的使用 </span>
<a href="https://ppt.cc/fTSM3x"><img src="https://ppt.cc/fTSM3x@.png" border="0" ></a>

In [ ]:
df_fs["董事長兼任總經理"]=0
df_fs.loc[arr1, "董事長兼任總經理"] = 1
df_fs.to_excel('上市櫃公司.xlsx', index=False)

### <span style="color : white">高階主管薪酬長期目標(Executive Compensation LT Objectives) :  0 or 1 </span>

#### 上市櫃高階主管薪酬資料 : 個體(A) 和合併(M)混合在一起
#### 發現1 : 經檢視，幾乎取個體即可 : 公司M值幾乎為0，但個體(A)會>0
#### 發現2 : 部分公司僅有A值或M值，不會2者都有
#### M值>0公司 : 5276達輝 (僅有M值)，8038 長園科 (A值=0，M值>0)

In [81]:
df_comLT_obj=file("2024年上市櫃高階主管薪酬.xlsx",sheet_name="Sheet1")
df_comLT_obj=df_comLT_obj[df_comLT_obj["合併(M)/個體(A)"]=="A"]
df_comLT_obj.to_excel('2024年上市櫃高階主管薪酬.xlsx', index=False)

#### 另外建立同時含有個體和合併的工作表，以完整包含公司名單

In [84]:
df_comLT_obj_AplusM=file("2024年上市櫃高階主管薪酬.xlsx",sheet_name="A+M")
with pd.ExcelWriter('2024年上市櫃高階主管薪酬.xlsx', engine='openpyxl') as writer:
    df_comLT_obj.to_excel(writer, index=False,sheet_name='Sheet1')
    df_comLT_obj_AplusM.to_excel(writer, index=False,sheet_name='A+M')
    

In [85]:
df_comLT_obj["代號"]=df_comLT_obj["代號"].astype(int)
df_comLT_obj_AplusM["代號"]=df_comLT_obj_AplusM["代號"].astype(int)
df_comLT_obj=df_comLT_obj.sort_values(by="代號", ascending=True)
df_comLT_obj_AplusM=df_comLT_obj_AplusM.sort_values(by="代號", ascending=True)
print(len(df_comLT_obj_AplusM["代號"].unique()))
print("高階主管薪酬相對上市櫃公司之缺失值: ",np.setdiff1d(df_fs["代號"].values.flatten(),df_comLT_obj_AplusM["代號"].values.flatten()))
print("上市櫃公司相對高階主管薪酬之缺失值: ",np.setdiff1d(df_comLT_obj_AplusM["代號"].values.flatten(),df_fs["代號"].values.flatten()))

1909
高階主管薪酬相對上市櫃公司之缺失值:  [  2491   2745   3537   4804   6788   6907   7717   7749   9105   9110
 911622]
上市櫃公司相對高階主管薪酬之缺失值:  [6620 6722 7777 7795]


### np.where() --> 確認高階主管薪酬相對上市櫃公司之缺失值的index
#### np.where(condition) --> 回傳值的索引
#### np.where(condition,True值下的結果，False值下的結果)

In [ ]:
df_comLT_obj_nonCommon_code=np.setdiff1d(df_fs["代號"].values.flatten(),df_comLT_obj_AplusM["代號"].values.flatten())
print(np.where(df_fs["代號"].values.flatten() == df_comLT_obj_nonCommon_code[:, None])[1])
df_comLT_obj_nonCommon_indices=np.where(df_fs["代號"].values.flatten() == df_comLT_obj_nonCommon_code[:, None])[1]
print(df_fs.loc[df_comLT_obj_nonCommon_indices,"名稱"])

[ 426  511  794 1036 1573 1627 1674 1687 1870 1871 1915]
426         吉祥全
511          五福
794          堡達
1036      大略-KY
1573        華景電
1627     雅特力-KY
1674    萊德光電-KY
1687      意騰-KY
1870     泰金寶-DR
1871     越南控-DR
1915     泰聚亨-DR
Name: 名稱, dtype: object


In [88]:
print(np.where(df_fs["代號"].values.flatten() == df_comLT_obj_nonCommon_code[:, None]))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 426,  511,  794, 1036, 1573, 1627, 1674, 1687, 1870, 1871, 1915]))


In [ ]:
df_comLT_obj["高階主管薪酬長期目標"]=np.where(df_comLT_obj["高階主管薪酬(含認股權)"]==0, 0, 1)
with pd.ExcelWriter('2024年上市櫃高階主管薪酬.xlsx', engine='openpyxl') as writer:
    df_comLT_obj.to_excel(writer, index=False,sheet_name='Sheet1')
    df_comLT_obj_AplusM.to_excel(writer, index=False,sheet_name='A+M')


In [ ]:
df_fs["高階主管薪酬長期目標"]=0
df_ECLTO_1=np.array(df_comLT_obj[df_comLT_obj["高階主管薪酬長期目標"]==1]["代號"])
print(df_ECLTO_1)
if len(np.setdiff1d(df_ECLTO_1, df_fs["代號"].values.flatten()))==0:
    print("無缺失值")
else:
    print("有缺失值")

df_ECLTO_1,arr1,arr2=np.intersect1d(df_ECLTO_1, df_fs["代號"].values.flatten(),return_indices=True)
df_fs.loc[arr2, "高階主管薪酬長期目標"] = 1
df_fs.to_excel('上市櫃公司.xlsx', index=False)

[1259 1440 1731 2308 2412 3045 3189 3287 3465 6123 6173 6435 6640 8255]
無缺失值


In [ ]:
print(df_fs["高階主管薪酬長期目標"].value_counts())

高階主管薪酬長期目標
0    1902
1      14
Name: count, dtype: int64


### Ownership : 企業是否由家族或政府控制、最終控制人控制權與現金流量權偏離比例

In [87]:
df_ownership=file("2025年上市櫃公司_所有權結構.xlsx",sheet_name="Sheet1")
df_ownership_control_code=df_ownership["集團控制型態"].unique()
print(f"以下有{len(df_ownership_control_code)}種集團控制代號: {df_ownership_control_code}")

以下有4種集團控制代號: ['F' 'A' 'M' 'G']


#### <span style="color : #F9C492"> map( functon ) = 根據輸入的映射關係或函數，將 Series 中的數值進行映射（轉換）</span>

####  <span style="color : #F9C492"> 方法2 : 用isin判斷M和M以外的值，回傳布林值 </span>

#### 結果 : M = 0 = 非由公股 or家族控制， A/F/G = 1 = 由家族OR公股控制

#### <a href="https://ppt.cc/fiPvpx"><img src="https://ppt.cc/fiPvpx@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>

In [88]:
df_ownership["代號"]=df_ownership["代號"].astype(int)
df_ownership=df_ownership.sort_values(by="代號", ascending=True)
df_ownership.to_excel("2025年上市櫃公司_所有權結構.xlsx",sheet_name="Sheet1")
df_ownership["集團控制型態_量化"]=df_ownership["集團控制型態"].map({"M":0,"A":1,"F":1,"G":1})
df_ownership.to_excel('2025年上市櫃公司_所有權結構.xlsx', index=False)

#### 6722 : 輝創電子

In [6]:
non_common_controlType_ownership=np.setdiff1d(df_fs["代號"].values.flatten(), df_ownership["代號"].values.flatten())
print("所有權結構模組相較上市櫃資料缺失值:",non_common_controlType_ownership)
non_common_controlType_df_fs=np.setdiff1d(df_ownership["代號"].values.flatten(), df_fs["代號"].values.flatten())
print("上市櫃資料相較所有權結構模組缺失值:",non_common_controlType_df_fs)

所有權結構模組相較上市櫃資料缺失值: [  9105   9110 911622]
上市櫃資料相較所有權結構模組缺失值: [3717 6620 6722 7777 7795]


#### df_fs.loc[arr1,"集團控制型態"]=df_ownership["集團控制型態"].values[arr2] 
##### <span style= "color: purple"> 若不加後面.values[arr2]則資料無法對齊--> 照理講會出錯，但會自動往上填，導致最後2格為空白 </span>

##### .set_index() = set columns as index

In [ ]:
df_ownership_companyCode=df_ownership.set_index("代號")["集團控制型態"].index
df_fs_ownership_1,arr1,arr2=np.intersect1d(df_fs["代號"].values.flatten(),df_ownership_companyCode,return_indices=True)
df_fs.loc[arr1,"集團控制型態"]=df_ownership["集團控制型態"].values[arr2] 
df_fs.loc[arr1,"集團控制型態_量化"]=df_ownership["集團控制型態_量化"].values[arr2]   
df_fs.loc[arr1,"控制持股%"]=df_ownership["控制持股%"].values[arr2]
df_fs.loc[arr1,"席次盈餘偏離差"]=df_ownership["席次盈餘偏離差"].values[arr2]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

###  <span style="color : white">控制變數 : AI </span>
####  <span style="color : white">個別企業產業類別屬於半導體業、電子零組件業、其他電子業、電器電纜業、電腦及周邊設備業時，則值為1</span>
#### <span style="color : white"> 前述產業以外之其他產業，則變數值為0</span>

#### First prompt: 加入證交所(TSE)產業名稱

In [85]:
df_industry=file("2025年上市櫃公司產業別.xlsx",sheet_name="Sheet1")
df_industry["代號"]=df_industry["代號"].astype(int).sort_values(ascending=True)
df_industry["TSE 產業別"]=df_industry["TSE 產業別"].astype(int)
df_industry.to_excel("2025年上市櫃公司產業別.xlsx",sheet_name="Sheet1")

df_industry_lost,arr1_ind,arr2_ind=np.intersect1d(df_industry["代號"].values.flatten(),df_fs["代號"].values.flatten(),return_indices=True)
df_fs.loc[arr2_ind,"TSE產業名稱"]=df_industry["TSE產業_名稱"].values[arr1_ind]
df_fs.loc[arr2_ind,"TSE 產業別"]=df_industry["TSE 產業別"].values[arr1_ind]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

In [44]:
print(df_fs["TSE產業名稱"].value_counts())
print(df_fs["TSE產業名稱"].unique())
print(f"總共有{len(df_fs['TSE產業名稱'].unique())}種產業")

TSE產業名稱
電子零組件       209
半導體業        193
生技醫療業       145
光電業         118
電腦及週邊設備業    110
電機機械         98
其他           95
其他電子業        92
通訊網路業        92
建材營造         86
紡織纖維         52
觀光餐旅         49
鋼鐵工業         49
資訊服務業        43
化學工業         42
金融業          40
汽車工業         38
電子通路業        36
綠能環保         35
數位雲端         34
食品工業         33
航運業          33
居家生活         32
運動休閒         26
文化創意業        26
塑膠工業         25
貿易百貨         19
電器電纜         16
油電燃氣業        12
橡膠工業         11
造紙工業          7
水泥工業          7
玻璃陶瓷          5
農業科技          4
存託憑證          3
電子工業          1
Name: count, dtype: int64
['水泥工業' '食品工業' '農業科技' '觀光餐旅' '塑膠工業' '建材營造' '汽車工業' '電子零組件' '其他' '紡織纖維'
 '運動休閒' '電機機械' '生技醫療業' '電腦及週邊設備業' '電器電纜' '化學工業' '其他電子業' '玻璃陶瓷' '造紙工業'
 '鋼鐵工業' '居家生活' '橡膠工業' '航運業' '半導體業' '通訊網路業' '光電業' '電子通路業' '資訊服務業' '貿易百貨'
 '油電燃氣業' '數位雲端' '金融業' '文化創意業' '綠能環保' '電子工業' '存託憑證']
總共有36種產業


#### Series.isin(values) --> 依據條件回傳布林值
#### Series.where(condition, True的值, False的值)

In [45]:
AI_1=np.array(["半導體業","電子零組件","其他電子業","電器電纜業","電腦及週邊設備業"])
df_fs["AI變數"]=np.where(df_fs["TSE產業名稱"].isin(AI_1), 1, 0)
df_fs.to_excel('上市櫃公司.xlsx', index=False)

#### <span style="color : #FFFBB1"> 分別產出僅含AI產業的資料庫，與僅含非AI產業的資料庫 </span>

In [48]:
df_fs_AI=df_fs[df_fs["AI變數"]==1]
print(f"總計有{len(df_fs_AI)}間公司包含在AI產業的定義內")
df_fs_AI.to_excel('上市櫃公司_AI.xlsx', index=False)

總計有604間公司包含在AI產業的定義內


In [49]:
df_fs_nonAI=df_fs[df_fs["AI變數"]==0]
print(f"總計有{len(df_fs_nonAI)}間公司不包含在AI產業的定義內")
df_fs_nonAI.to_excel('上市櫃公司_nonAI.xlsx', index=False)

總計有1312間公司不包含在AI產業的定義內


### <span style="color:#3BC1A8"> 1/22 task : 找尋TEJ 未動用信用額度相關資料 </span>

#### 結果 : TEJ未提供未動用信用額度相關資訊，改用現金充裕度代替

<a href="https://ppt.cc/fAJAMx"><img src="https://ppt.cc/fAJAMx@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>

In [83]:
df_cash_adequacy=file("上市櫃公司_現金充裕度(2023).xlsx",sheet_name="Sheet1")
df_cash_adequacy["TCRI"]=df_cash_adequacy["TCRI"].astype(str).str.replace('-',"0").str.replace("D","0").str.replace('C',"0").astype(int)
df_cash_adequacy=df_cash_adequacy.sort_values(by="代號", ascending=True)
df_cash_adequacy.to_excel("上市櫃公司_現金充裕度(2023).xlsx",sheet_name="Sheet1")
df_cash_adequacy_common,arr1_ind,arr2_ind=np.intersect1d(df_fs["代號"].values.flatten(), df_cash_adequacy["代號"].values.flatten(),return_indices=True)
df_fs.loc[:,"現金充裕度"]=0
df_fs.loc[arr1_ind,"現金充裕度"]=df_cash_adequacy["現金充裕度"].values[arr2_ind].astype(float)
df_fs.loc[arr1_ind,"TCRI"]=df_cash_adequacy["TCRI"].values[arr2_ind].astype(int)
df_fs.to_excel('上市櫃公司.xlsx', index=False)

In [64]:
print(df_fs["現金充裕度"].describe())
print(df_fs["TCRI"].describe())


count     1914.000000
mean        58.446102
std        515.265092
min         -8.410000
25%          3.632500
50%          9.050000
75%         24.097500
max      16876.440000
Name: 現金充裕度, dtype: float64
count     1916
unique      10
top          6
freq       575
Name: TCRI, dtype: int64


## <span style = "color : #3F9AAE"> H2模型 : 集團與非集團分類 </style>
### 原先想法 : 判定標準取自中華徵信所全方位企業資料庫中臺灣地區 300 家大型集團及 250 家中集型集團之名單
### <span style = "color : #beda24">--> 結果 : 學校沒買資料庫...... </span>
### 更改分類方法 : 以TEJ之集團名稱作為分類標準 (集團名稱 = 最終控制者 )
##### 圖一 : 股權結構(控制持股與董監結構) 欄位說明
##### <a href="https://ppt.cc/fyV5bx"><img src="https://ppt.cc/fyV5bx@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>

##### 圖2 : 以1301 台塑集團為例下的分類
##### <a href="https://ppt.cc/fV2F5x"><img src="https://ppt.cc/fV2F5x@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>

#### First prompt : 將集團名稱資料導入上市櫃資料中

In [71]:
df_group=file("2025年_上市櫃公司集團.xlsx",sheet_name="Sheet1")
df_group["代號"]=df_group["代號"].astype(int)
df_group=df_group.sort_values(by="代號",ascending=True)
df_group.to_excel("2025年_上市櫃公司集團.xlsx",index=False)
df_group_common,arr1,arr2=np.intersect1d(df_fs["代號"].values.flatten(),df_group["代號"].values.flatten(),return_indices=True)
df_fs.loc[arr1,"集團名稱"]=df_group["集團名稱"].values[arr2]
df_fs.to_excel('上市櫃公司.xlsx', index=False)

In [77]:
df_group_non=np.setdiff1d(df_fs["代號"],df_group["代號"])
df_group_non_fs=np.setdiff1d(df_group["代號"],df_fs["代號"])
print(df_group_non_fs)
print(df_group_non)

[1623 3717 6620 6722 6961 7777 7795 7823]
[  9105   9110 911622]


#### 2nd prompt : 檢測那些上市櫃公司非屬集團
#### 將集團次數計算後，另外匯出excel檔
#### 暫定標準 : 僅出現一次之集團視為非集團企業

In [ ]:
df_fs_GroupCount=df_fs["集團名稱"].value_counts()
df_fs_GroupCount=file("集團計算.xlsx",sheet_name="df_fs_GroupCount")
print(df_fs_GroupCount.head())

           集團名稱  count
0  2352  明基友達集團     27
1    6241  台鋼集團     19
2    2317  鴻海集團     18
3  1605  華新麗華集團     15
4    1301  台塑集團     10


In [ ]:
df_fs_GroupCount_1time=df_fs_GroupCount[df_fs_GroupCount["count"]==1]
# 1123/1903
print(f"僅在上市櫃市場中出現一次的公司有{len(df_fs_GroupCount_1time)}個")

# 會回傳布林值
df_fs_GroupCount_1time_Group=df_fs_GroupCount_1time["集團名稱"].str.contains("集團",regex=True)

僅在上市櫃市場中出現一次的公司有1123個


### 另外設立代號欄將原先的集團名稱數字化
#### 使用 Regex 萃取出儲存格內的數字
#### e.g : 2352  明基友達集團 --> 2352
#### <span style="color: #beda24"> 首行先使用r'^(\w+)' --> \w=數字、字母、or底線字元   +=字串中的字元可以出現無限次 ^=指定字串必須在開始處 </span>

#### <span style="color: #cc17e4"> 第二行使用r'\D+' --> \D=非數字字元 \d=數字字元 
#### e.g : 58DOF 三地集團 = 58
#### str.replace ()--> 偵測到字母後全部用空白取代後，再轉成數字型態
##### <a href="https://ppt.cc/fM9TIx"><img src="https://ppt.cc/fM9TIx@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>


In [ ]:
print(type(df_fs_GroupCount_1time_Group[248]))
df_fs_GroupCount_1time_Group_False=df_fs_GroupCount_1time["集團名稱"][df_fs_GroupCount_1time_Group==np.bool(0)].index
df_fs_GroupCount.to_excel("集團計算.xlsx",sheet_name="df_fs_GroupCount")
# 可用str.split()[0]代替 : 資料結構為 2352 明基友達集團 --> 
df_fs_GroupCount["代號"]=df_fs_GroupCount["集團名稱"].str.extract(r'^(\w+)')
df_fs_GroupCount["代號"]=df_fs_GroupCount["代號"].str.replace(r'\D+',"",regex=True).astype(int)
df_fs_GroupCount.to_excel("集團計算.xlsx",sheet_name="df_fs_GroupCount")

<class 'numpy.bool'>


### <span style="color: #3de8ba"> regex用法 </span>
#### .(點)：代表任意字元（包含數字、空白、中文字） 
#### * (星號)：代表前面的東西可以出現無限次 
#### 集團：最後必須剛好是這兩個字結尾
#### str.fullmatch( condition, True value, False Value )--> 針對每筆資料是否完全符合條件，回傳布林值

### 構想 : 分類集團與非集團企業 (直接在上市櫃公司.xlsx做分類)
#### 集團企業 = 1，非集團企業 = 0
#### 1.先直接將value_counts()超過 1 的公司直接當作集團企業
#### 2.把集團記數 = 1 的企業，再用文字探勘拆分成 <span style="color:red"> 含(不含)</span>集團的企業 
#### 3. count=1 企業且含<span style="color:red">"集團"</span>的企業=1，不含=0

#### <span style="color:#beda24">group_count_moreThan1 = 擁有超過一間上市櫃公司的集團名單 </span>

In [169]:
group_count_moreThan1=df_fs_GroupCount[df_fs_GroupCount["count"]>1]["集團名稱"]
print(f"總計有{len(group_count_moreThan1)}個集團擁有超過1間上市櫃公司")
print(type(group_count_moreThan1))


總計有248個集團擁有超過1間上市櫃公司
<class 'pandas.core.series.Series'>


#### 集團/非集團企業(量化) --> 集團企業=1 ，非集團企業=0
#### 步驟1: 先偵測<span style="color: green">集團名稱欄位</span>包含"集團"二字的企業歸類為集團，其餘為非集團
#### 步驟2: 再將剩餘未偵測出"集團"2字，但擁有<span style="color: orange">複數</span>上市櫃公司之企業歸類為集團企業

In [ ]:
# 應可用df_fs["集團名稱"].str.contains("集團") 代替  df_fs["集團名稱"].str.fullmatch(r'.*集團')
df_fs["集團/非集團企業(量化)"]=np.where(df_fs["集團名稱"].str.fullmatch(r'.*集團'),1,0)
list_group_count_moreThan1=list(group_count_moreThan1)

#有更有效率的寫法
for i in range(len(df_fs["集團名稱"])):
    if df_fs.loc[i,"集團名稱"] in list_group_count_moreThan1:
        df_fs.loc[i,"集團/非集團企業(量化)"]=1

df_fs.to_excel("上市櫃公司.xlsx",sheet_name="Sheet1")

In [194]:
df_fs["集團/非集團企業(標籤)"]=df_fs["集團/非集團企業(量化)"].map({0:"非集團",1:"集團"})
df_fs.to_excel("上市櫃公司.xlsx",sheet_name="Sheet1")

In [197]:
grouplist=df_fs[df_fs["集團/非集團企業(標籤)"]=="集團"][["代號","名稱","集團名稱"]]
nongroup_list=df_fs[df_fs["集團/非集團企業(標籤)"]=="非集團"][["代號","名稱","集團名稱"]]
grouplist.to_excel("集團企業名單.xlsx")
nongroup_list.to_excel("非集團企業名單.xlsx")